<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=4ae9d908aa64c564c1eacea4f8496189bfcc4d539ae937daa486db474219fc35
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-21 15:20:01
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -1.39 L (-0.96%)
Current PnL: -22.55 L (-14.78%)
CY Booked + Current PnL: -8.66 L (-5.67%)
-------------------
Total profit:  1.53 L
Total loss:  -24.08 L
-------------------
Total Booked + Current PnL: 18.32 L (14.58%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.96 L (61.41%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.52%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,0.01,-14.21,16.68,0.09,14420.0,-14324.0,86453.0,100.64,49.0,M-SC,4.15,253.0,-0.99,0.60,13.58,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.0,-2.03,-9.04,9.97,0.02,15829.0,-15789.0,158770.0,142.19,71.0,M-SC,4.20,234.0,-1.00,1.11,38.10,OX40N,NTT,PAINTS
83,VOLTAS,1530.0,-1.17,9.15,9.65,19.69,20197.0,17553.0,209295.0,-1.49,54.0,X-MC,3.57,78.0,0.87,1.46,16.57,XY25,NTT,AC
43,ITC,452.0,0.62,-0.32,10.80,10.44,21545.0,-650.0,199488.0,-40.34,50.0,X-LC,0.96,5.0,-0.03,1.39,5.22,X40,NTT,FMCG
53,NESTLEIND,1377.0,0.13,12.92,7.51,21.40,22508.0,34281.0,299707.0,4.80,63.0,X-LC,7.96,12.0,1.52,2.09,20.42,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SURYODAY,216.0,3.60,-19.49,52.81,23.03,76140.0,-34894.0,144177.0,57.89,49.0,H-SC,11.70,167.0,-0.46,1.01,43.05,XR,NTT,BANKS
40,INDUSINDBK,1800.0,2.01,-37.54,112.75,32.89,54374.0,-28982.0,48225.0,-695.74,64.0,L-MC,5.37,259.0,-0.53,0.34,32.81,XR,NTT,BANKS
52,MEDANTA,1486.0,1.15,2.45,21.11,24.08,27456.0,3112.0,130062.0,-5.81,44.0,X-SC,4.51,89.0,0.11,0.91,22.55,XY24,NTT,HEALTHCARE
14,CAMPUS,393.0,1.12,-11.53,50.66,33.28,75192.0,-19351.0,148424.0,-28.70,36.0,M-SC,2.96,222.0,-0.26,1.04,15.04,XY24,NTT,FOOTWEAR
79,UNITDSPR,1644.0,0.80,4.87,15.16,20.77,37225.0,11403.0,245547.0,-4.29,58.0,X-MC,3.36,62.0,0.31,1.71,12.13,X40N,NTT,BREWERIES


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-4.11,-34.90,73.42,12.90,104441.0,-76245.0,142251.0,109.13,59.0,H-SC,15.12,161.0,-0.73,0.99,18.28,OX40N,NTT,FINANCE
72,TANLA,1943.92,-3.48,-34.76,236.76,119.69,418208.0,-94123.0,176638.0,-34.29,30.0,H-SC,12.48,156.0,-0.23,1.23,39.54,AR,ATH,IT
56,QUESS,424.00,-3.25,-29.66,103.08,42.84,47131.0,-19283.0,45723.0,-53.72,35.0,X-SC,37.18,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
48,KPIGREEN,730.66,-3.21,-9.60,62.55,46.95,70853.0,-12023.0,113274.0,-16.60,30.0,H-SC,7.38,154.0,-0.17,0.79,41.91,MH,ATH,POWER
47,KANSAINER,340.00,-2.94,-22.32,46.09,13.47,96543.0,-60201.0,209466.0,-68.12,27.0,H-SC,1.64,158.0,-0.62,1.46,6.51,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-2.08,0.64,105.41,106.72,130841.0,790.0,124126.0,-50.25,36.0,H-SC,4.42,139.0,0.01,0.87,24.17,AR,ATH,MISC
85,WIPRO,420.00,-0.76,0.24,71.85,72.27,122043.0,410.0,169858.0,-13.79,52.0,M-LC,5.06,99.0,0.00,1.19,6.43,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.18,-3.94,116.45,107.91,165151.0,-5823.0,141821.0,-21.77,36.0,M-SC,10.20,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.03,-9.04,9.97,0.02,15829.0,-15789.0,158770.0,142.19,71.0,M-SC,4.20,234.0,-1.00,1.11,38.10,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.01,-14.21,16.68,0.09,14420.0,-14324.0,86453.0,100.64,49.0,M-SC,4.15,253.0,-0.99,0.60,13.58,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.94,-22.32,46.09,13.47,96543.0,-60201.0,209466.0,-68.12,27.0,H-SC,1.64,158.0,-0.62,1.46,6.51,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.10,-23.23,30.25,-0.01,29679.0,-29693.0,98112.0,19986.96,63.0,M-SC,1.54,233.0,-1.00,0.68,11.10,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.76,0.24,71.85,72.27,122043.0,410.0,169858.0,-13.79,52.0,M-LC,5.06,99.0,0.00,1.19,6.43,XR,NTT,IT
38,INDIAMART,4810.62,-2.08,0.64,105.41,106.72,130841.0,790.0,124126.0,-50.25,36.0,H-SC,4.42,139.0,0.01,0.87,24.17,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-2.08,0.64,105.41,106.72,130841.0,790.0,124126.0,-50.25,36.0,H-SC,4.42,139.0,0.01,0.87,24.17,AR,ATH,MISC
85,WIPRO,420.00,-0.76,0.24,71.85,72.27,122043.0,410.0,169858.0,-13.79,52.0,M-LC,5.06,99.0,0.00,1.19,6.43,XR,NTT,IT
25,FINCABLES,1641.55,-1.18,-3.94,116.45,107.91,165151.0,-5823.0,141821.0,-21.77,36.0,M-SC,10.20,216.0,-0.04,0.99,0.00,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.50,-5.27,92.05,81.92,100099.0,-6054.0,108744.0,-44.55,32.0,M-SC,6.18,236.0,-0.06,0.76,28.74,XR,NTT,DURABLES
86,ZYDUSLIFE,1286.17,-0.39,-2.53,39.14,35.63,79956.0,-5293.0,204281.0,-16.30,32.0,H-MC,4.76,119.0,-0.07,1.43,13.73,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.83,-35.43,109.39,35.20,90884.0,-45587.0,83083.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,-1.57,-16.16,70.94,43.32,96876.0,-26312.0,136560.0,-27.94,22.0,X-SC,1.72,90.0,-0.27,0.95,4.85,X40N,ATH,ELECTRICAL
76,TMPV,600.00,0.69,-14.15,65.61,42.18,155696.0,-39104.0,237306.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.69,XY24,NTT,AUTO
46,JSWINFRA,342.00,-2.32,-9.08,26.74,15.23,48707.0,-18198.0,182149.0,-22.32,25.0,X-MC,6.38,66.0,-0.37,1.27,20.24,X40N,NTT,REALTY
26,GILLETTE,11206.78,-1.51,-2.19,35.39,32.42,87882.0,-5571.0,248325.0,-19.56,29.0,X-SC,6.30,85.0,-0.06,1.73,12.57,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,0.69,-14.15,65.61,42.18,155696.0,-39104.0,237306.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.69,XY24,NTT,AUTO
43,ITC,452.00,0.62,-0.32,10.80,10.44,21545.0,-650.0,199488.0,-40.34,50.0,X-LC,0.96,5.0,-0.03,1.39,5.22,X40,NTT,FMCG
36,ICICIPRULI,790.00,-1.16,1.33,29.24,30.97,52368.0,2358.0,179096.0,-22.52,49.0,X-MC,1.49,75.0,0.05,1.25,14.06,X40,ATH,INSURANCE
34,HONAUT,58357.33,-1.57,-16.16,70.94,43.32,96876.0,-26312.0,136560.0,-27.94,22.0,X-SC,1.72,90.0,-0.27,0.95,4.85,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-0.86,-22.86,112.80,64.15,207044.0,-54401.0,183550.0,-55.00,45.0,X-SC,1.83,82.0,-0.26,1.28,2.96,XY24,BTT,CEMENT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.83,-35.43,109.39,35.20,90884.0,-45587.0,83083.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
56,QUESS,424.00,-3.25,-29.66,103.08,42.84,47131.0,-19283.0,45723.0,-53.72,35.0,X-SC,37.18,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
9,BAJAJHFL,181.50,-0.62,-13.50,71.63,48.45,124228.0,-27076.0,173430.0,-19.66,30.0,X-MC,7.28,64.0,-0.22,1.21,0.01,X40N,ATH,FINANCE
54,PAGEIND,51605.07,0.67,-5.15,32.92,26.07,51125.0,-8440.0,155300.0,-30.18,35.0,X-MC,8.69,55.0,-0.17,1.08,0.67,X40,ATH,APPARELS
76,TMPV,600.00,0.69,-14.15,65.61,42.18,155696.0,-39104.0,237306.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.69,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-3.25,-29.66,103.08,42.84,47131.0,-19283.0,45723.0,-53.72,35.0,X-SC,37.18,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-1.06,-46.92,186.93,52.31,144033.0,-68108.0,77052.0,-43.47,30.0,X-SC,3.43,91.0,-0.47,0.54,3.14,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.83,-35.43,109.39,35.20,90884.0,-45587.0,83083.0,8.88,22.0,X-SC,12.02,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,1.15,2.45,21.11,24.08,27456.0,3112.0,130062.0,-5.81,44.0,X-SC,4.51,89.0,0.11,0.91,22.55,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-2.14,-3.76,26.34,21.58,35523.0,-5274.0,134865.0,-51.46,35.0,X-MC,7.21,56.0,-0.15,0.94,19.40,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.16,-14.42,39.38,19.28,116593.0,-49884.0,296072.0,-25.24,63.0,X-LC,7.77,1.0,-0.43,2.07,9.45,X40,ATH,IT
41,INFY,2275.00,-0.30,4.60,48.06,54.87,157288.0,14386.0,327274.0,-18.52,58.0,X-LC,3.42,2.0,0.09,2.28,13.26,X40,BTT,IT
76,TMPV,600.00,0.69,-14.15,65.61,42.18,155696.0,-39104.0,237306.0,-23.08,24.0,X-LC,0.10,3.0,-0.25,1.66,0.69,XY24,NTT,AUTO
82,VBL,671.64,-0.82,-9.54,49.99,35.67,142819.0,-30146.0,285696.0,-20.30,41.0,X-LC,6.13,4.0,-0.21,1.99,2.99,X40N,ATH,FMCG
43,ITC,452.00,0.62,-0.32,10.80,10.44,21545.0,-650.0,199488.0,-40.34,50.0,X-LC,0.96,5.0,-0.03,1.39,5.22,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-1.57,-12.68,108.05,81.67,88511.0,-11893.0,81917.0,7366.67,52.0,L-SC,18.40,271.0,-0.13,0.57,58.64,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.79,-12.87,60.16,39.54,53024.0,-13020.0,88139.0,-31.83,66.0,M-SC,6.88,220.0,-0.25,0.62,24.28,AR,ATH,AUTO
51,MASFIN,398.61,-0.55,-3.06,25.90,22.05,24600.0,-3000.0,94980.0,-16.44,57.0,H-SC,7.05,164.0,-0.12,0.66,38.06,XR,ATH,FINANCE
70,SURYODAY,216.00,3.60,-19.49,52.81,23.03,76140.0,-34894.0,144177.0,57.89,49.0,H-SC,11.70,167.0,-0.46,1.01,43.05,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-2.03,-9.04,9.97,0.02,15829.0,-15789.0,158770.0,142.19,71.0,M-SC,4.20,234.0,-1.00,1.11,38.10,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.03,-9.04,9.97,0.02,15829.0,-15789.0,158770.0,142.19,71.0,M-SC,4.20,234.0,-1.00,1.11,38.10,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.56,0.52,20.32,20.95,51426.0,1311.0,253079.0,-0.49,76.0,X-LC,12.12,31.0,0.03,1.77,36.34,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,2.01,-37.54,112.75,32.89,54374.0,-28982.0,48225.0,-695.74,64.0,L-MC,5.37,259.0,-0.53,0.34,32.81,XR,NTT,BANKS
13,BSOFT,831.70,-1.95,-30.89,118.18,50.79,114878.0,-43447.0,97206.0,-11.48,53.0,H-SC,7.66,151.0,-0.38,0.68,13.18,XR,ATH,IT
68,SONACOMS,806.63,-0.79,-12.87,60.16,39.54,53024.0,-13020.0,88139.0,-31.83,66.0,M-SC,6.88,220.0,-0.25,0.62,24.28,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.74
1,25,44.59
2,50,75.36


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.60
MC    29.27
LC    25.12
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.89
X40      21.14
X40N     11.84
XR        9.65
XY25      9.13
AR        9.01
OX40N     7.69
X200      1.86
SR        0.99
MH        0.79
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.02
X-MC    22.84
X-LC    19.98
M-SC    12.05
X-SC     8.13
H-MC     4.71
H-LC     2.97
L-SC     1.40
M-MC     1.38
M-LC     1.19
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.75,-4.81,38.71
IT,13.79,-14.46,73.54
FINANCE,9.44,-14.52,62.91
MISC,7.44,-22.07,72.07
PAINTS,5.98,-7.90,23.92
ELECTRICAL,5.88,-11.37,52.16
INSURANCE,4.42,0.16,35.02
PHARMA,3.86,-1.16,33.54
AUTO,3.35,-20.15,71.67


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3041559.0,21
AR,1284054.0,10
XR,1280678.0,13
X40,1008779.0,14
X40N,750981.0,9
OX40N,690208.0,10
XY25,333996.0,6
SR,277618.0,2
MH,70853.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3538093.0,25
M-SC,1333209.0,15
X-MC,1208926.0,16
X-LC,899793.0,11
X-SC,755293.0,8
H-MC,404388.0,3
L-SC,260211.0,3
H-LC,132093.0,2
M-LC,122043.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1216442.0      6
           AR         870442.0      5
           XR         783530.0      7
M-SC       XY24       778731.0      6
X-MC       X40        439736.0      7
X-LC       X40        390277.0      5
X-MC       XY24       348543.0      3
H-SC       OX40N      319208.0      4
X-SC       X40N       294896.0      3
M-SC       OX40N      283012.0      5
X-SC       XY24       281631.0      3
H-SC       SR         277618.0      2
X-MC       X40N       245683.0      4
X-LC       XY24       227581.0      2
H-MC       AR         215757.0      2
X-LC       X40N       210402.0      2
H-MC       XY24       188631.0      1
X-SC       X40        178766.0      2
X-MC       XY25       174964.0      2
L-SC       XR         172223.0      2
M-SC       XR         148508.0      2
           AR         122958.0      2
M-LC       XR         122043.0      1
L-SC       OX40N       87988.0      1
H-LC       AR          74897.0      1
X-LC       XY25        71533.0      2
H-SC       MH          70853.0      1
H-LC       X200        57196.0      1
L-MC       XR          54374.0      1
M-MC       XY25        48052.0      1
L-LC       XY25        39447.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 63.0 seconds
